1-a. MNIST 데이터를 훈련, 검증, 테스트 데이터로 나눈다.

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)

In [3]:
X,y = mnist['data'], mnist['target']

In [4]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

1-b. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시킨다. (n_estimators=100, random_state=42)

In [5]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)

In [6]:
from sklearn.ensemble import ExtraTreesClassifier
xt_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)

In [26]:
from sklearn.svm import LinearSVC # 선형
svm_clf = LinearSVC(max_iter=1000,tol=20,random_state=42)

In [12]:
from sklearn.neural_network import MLPClassifier
mlp_clf = MLPClassifier(random_state=42)

In [13]:
rnd_clf.fit(X_train, y_train)
xt_clf.fit(X_train, y_train)
mlp_clf.fit(X_train, y_train)

MLPClassifier(random_state=42)

In [27]:
svm_clf.fit(X_train, y_train)

LinearSVC(random_state=42, tol=20)

1-c. 이들을 직접 투표 분류기를 사용하는 앙상블로 연결한다.

In [28]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
    estimators=[('rnd', rnd_clf), ('svc', svm_clf), ('mlp', mlp_clf), ('xt', xt_clf)],
    voting ='hard'
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rnd', RandomForestClassifier(random_state=42)),
                             ('svc', LinearSVC(random_state=42, tol=20)),
                             ('mlp', MLPClassifier(random_state=42)),
                             ('xt', ExtraTreesClassifier(random_state=42))])

In [29]:
from sklearn.metrics import accuracy_score
for clf in (rnd_clf, svm_clf, mlp_clf, voting_clf):
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

RandomForestClassifier 0.9705
LinearSVC 0.865
MLPClassifier 0.9619
VotingClassifier 0.9707


1-d. 다른 분류기보다 성능이 많이 떨어졌던 선형 SVM 제거 후 VotingClassifier를 다시 평가

In [30]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
    estimators=[('rnd', rnd_clf), ('mlp', mlp_clf), ('xt', xt_clf)],
    voting ='hard'
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rnd', RandomForestClassifier(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42)),
                             ('xt', ExtraTreesClassifier(random_state=42))])

In [31]:
from sklearn.metrics import accuracy_score
for clf in (rnd_clf, mlp_clf, voting_clf):
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

RandomForestClassifier 0.9705
MLPClassifier 0.9619
VotingClassifier 0.9743


1-d. 간접 투표 분류기를 사용하여 앙상블로 연결.

In [32]:
svm_clf = SVC(probability=True)

voting_clf = VotingClassifier(
    estimators=[('rnd', rnd_clf), ('mlp', mlp_clf), ('xt', xt_clf)],
    voting ='soft'
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rnd', RandomForestClassifier(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42)),
                             ('xt', ExtraTreesClassifier(random_state=42))],
                 voting='soft')